In [1]:
import bqplot.pyplot as plt
import pandas as pd
from bqplot.traits import convert_to_date
from bqplot import ColorScale
import datetime
import pickle
import numpy as np

In [2]:
features = ['CAPE', "BAAFFM", "BCI", "CCI", "CLI", "IJC", "PMIC", "T10Y2Y", "T10Y3M"]
model_names_classification = ["Logistic Regression", "Penalized SVM", "Random Forest", "LSTM"]
model_names_regression = ["Linear Regression", "LSTM"]
target_variables_classification = ["USRECD"]
target_variables_regression = ["GDP_rate"]

# Overview

We explored two different tasks, classification and regression.

For the classification part, the task is to predict whether the US would be in a recession 180 days later, using the current features. Our models achieved considerable performances in this task.

For the regression part, the task is to predict the exact GDP growth rate and the exact GDP value. Our models' performances were not that good, so we won't focus on them.

# Comparisons of model performance

## Validation

We used TimeSeriesSplit with 3 splits to validate the performances of our models. The following graph displays comparisons between different models.

In [3]:
plt.figure()
plt.title('Average AUC ROC, USRECD')
x = ['Logistic Regression', 'Penalized SVM', 'Random Forest', 'LSTM']
y = [0.831, 0.760, 0.825, 0.823]
plt.bar(x, y)
plt.show()

In [4]:
plt.figure()
plt.title('Average accuracy, USRECD')
x = ['Logistic Regression', 'Penalized SVM', 'Random Forest', 'LSTM']
y = [0.899, 0.595, 0.906, 0.906]
plt.bar(x, y)
plt.show()

In [5]:
plt.figure()
plt.title('R2 Score, GDP growth rate')
x = ['Linear Regression', 'LSTM']
y = [-1.514, -0.463]
plt.bar(x, y)
plt.show()

## Test

We tested our models' performances on a test set, which was never used for training. The following graph displays comparisons between different models. For the purpose of comparisons, we also included results from naive methods.

In [6]:
plt.figure()
plt.title('AUC ROC, USRECD')
x = ['Logistic Regression', 'Penalized SVM', 'Random Forest', 'LSTM', 'Baseline #1', 'Baseline #2', 'Baseline #3']
y = [0.948, 0.918, 0.923, 0.743, 0.5, 0.508, 0.5]
plt.bar(x, y)
plt.show()

In [7]:
plt.figure()
plt.title('Accuracy, USRECD')
x = ['Logistic Regression', 'Penalized SVM', 'Random Forest', 'LSTM', 'Baseline #1', 'Baseline #2', 'Baseline #3']
y = [0.963, 0.963, 0.963, 0.962, 0.963, 0.871, 0.963]
plt.bar(x, y)
plt.show()

In [8]:
plt.figure()
plt.title('R2 Score, GDP growth rate')
x = ['Linear Regression', 'LSTM']
y = [0.350, -0.067]
plt.bar(x, y)
plt.show()

In [9]:
def plot(model_name, target_variable):
    predictions_path = f'../predictions/{model_name}_{target_variable}.pkl'
    predictions = pickle.load(open(predictions_path, 'rb'))
    plt.figure()
    plt.title(model_name + ' ' + target_variable)
    plt.plot(x=dates, y=df[target_variable], colors='blue')
    plt.plot(x=convert_to_date(extended_dates, fmt='%Y-%m-%d'), y=predictions, colors='red')
    plt.show()